In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename

In [135]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET]

train_paths = glob('../feature/raw_main/*_train.gz')
test_paths = glob('../feature/raw_main/*_test.gz')

train_paths = [path for path in train_paths 
               if path.count('DT') 
#                or path.count('day_no') 
               or path.count('Amt')
               or path.count('Fraud') 
               or path.count('D')
               or path.count('C')
               or path.count('V')
#                or path.count('card')
#                or path.count('addr')
#                or path.count('Reg')
#                or path.count('P_email')
#                or path.count('R_email')
#                or path.count('M')
#                or path.count('Product')
              ]
test_paths = [path for path in test_paths 
               if path.count('DT') 
#                or path.count('day_no') 
               or path.count('Amt')
               or path.count('Fraud') 
               or path.count('D')
               or path.count('C')
               or path.count('V')
#                or path.count('card')
#                or path.count('addr')
#                or path.count('Reg')
#                or path.count('P_email')
#                or path.count('R_email')
#                or path.count('M')
#                or path.count('Product')
              ]

train_df = parallel_load_data(train_paths)
test_df = parallel_load_data(test_paths)

In [31]:
train_df.head()

,D15,V7,V210,V225,V203,V3,V61,V252,TransactionID,V170,...,V304,V272,D13,V267,V303,TransactionAmt,V91,D8,V2,ProductCD
0,0.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,2987000,NaN,...,0.0,NaN,NaN,NaN,0.0,68.5,0.0,NaN,1.0,W
1,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2987001,NaN,...,0.0,NaN,NaN,NaN,0.0,29.0,0.0,NaN,NaN,W
2,315.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,2987002,NaN,...,0.0,NaN,NaN,NaN,0.0,59.0,0.0,NaN,1.0,W
3,111.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2987003,NaN,...,0.0,NaN,NaN,NaN,0.0,50.0,0.0,NaN,NaN,W
4,NaN,NaN,0.0,0.0,0.0,NaN,NaN,1.0,2987004,1.0,...,1.0,0.0,NaN,0.0,1.0,50.0,NaN,NaN,NaN,H


In [32]:
train_df.set_index(COLUMN_ID, inplace=True)
list_path = glob('../output/0830_ieee__same_user__pattern-user_keys__card*')
for path in list_path:
    fname = path[56:].replace('csv', '').replace(r'.', '')
    same_user = pd.read_csv(path).set_index(COLUMN_ID)
    train_df[f'same_user__{fname}']  = same_user['same_user_id']

In [38]:
startdate = datetime.datetime(2017,11,4)
train_df['datetime'] = train_df['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
train_df['month'] = train_df['datetime'].map(lambda x: x.month)
train_df['date'] = train_df['datetime'].map(lambda x: x.date())

In [45]:
cols_V = sorted([int(col[1:]) for col in train_df.columns if col.startswith('V')])
cols_V = ['V' + str(col) for col in cols_V]
cols_D = sorted([int(col[1:]) for col in train_df.columns if col.startswith('D')])
cols_D = ['D' + str(col) for col in cols_D]
cols_C = sorted([int(col[1:]) for col in train_df.columns if col.startswith('C')])
cols_C = ['C' + str(col) for col in cols_C]
cols_same = sorted([col for col in train_df.columns if col.startswith('same')])
cols_trans = sorted([col for col in train_df.columns if col.startswith('Trans')])
cols_sort = cols_same + ['datetime', COLUMN_TARGET, 'ProductCD'] + cols_trans + cols_D + cols_C + cols_V

same_userの情報を使い、複数トランザクションをもつであろうユーザーの詳細を見ることで、C系やV系の特徴を掴む

In [125]:
# tmp = train_df.groupby('same_user__addr_pemail_M')['D1'].count()
# multi_user = tmp[tmp>3]
# multi_user_idx = multi_user.index
# train_df.sort_values(by=['same_user__addr_pemail_M', COLUMN_DT], inplace=True)
# train_df.reset_index(inplace=True)
# train_df.set_index('same_user__addr_pemail_M', inplace=True)
# train_df.loc[multi_user_idx][cols_sort].head(10000).to_csv('../output/0831_ieee__eda_same_user_feature_row10000.csv')
tmp = train_df.loc[multi_user_idx]
tmp[tmp[COLUMN_TARGET]==1].loc[multi_user_idx][cols_sort].head(10000).to_csv('../output/0831_ieee__eda_same_user_feature_FRAUD_row10000.csv')